<a href="https://colab.research.google.com/github/jasmineA20/MathModeling/blob/main/Math380Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a linear optimization approach to the scheduling problem. 

The decision variables are : \\
i : Couse + Section \\
j : Room    \\
k : Day of the week  \\
l : Time slot of the day  \\

Additional Variables to Consider: \\
N : Number of students registered for the course \\
C : Course capacity (Considering size of the room and number of students the instructor can teach)    


Variables to consider for Constraints: 

Objective Function \\

max $\sum_{i \in I}$  $\sum_{j \in J}$ $\sum _{k \in K}$ $\sum_ {l \in L} $ $x_{ijkl}$ * $\frac{N}{C} $ 

Constraints: 

Capacity: \\
Day : \\
Time slot and room constant: \\
Same slot everyday for classes that run for multiple days : \\

In [40]:
import numpy as np
import pandas as pd  
import cvxpy
import math 

In [41]:
def stage_one(prof_course_teaching_availability): 
  selection = cvxpy.Variable(prof_course_teaching_availability.shape, boolean=True)
  one_course_per_prof_constraint = cvxpy.sum(selection, axis=0) == 1 
  min_courses_constraint = cvxpy.sum(selection, axis=1) >= 1
  max_courses_constraint = cvxpy.sum(selection, axis=1) <= 3
  constraints = [one_course_per_prof_constraint, min_courses_constraint, max_courses_constraint]
  cost = cvxpy.sum(cvxpy.multiply(prof_course_teaching_availability, selection))
  problem = cvxpy.Problem(cvxpy.Maximize(cost), constraints=constraints)
  opt_cost = problem.solve(solver=cvxpy.GLPK_MI)

  return problem, opt_cost, selection.value

In [42]:
def stage_two(stage_one_res, time_slot_costs, course_costs, preference_weights, s2_type='baseline'):
  selection  = cvxpy.Variable((time_slot_costs.shape), boolean=True)
  constraints = [] 

  prof_count = stage_one_res.shape[0]
  course_count = stage_one_res.shape[1]

  time_slot_count = time_slot_costs.shape[1] 
  

  # Regular Constraints 
  one_for_each_course_constraint = cvxpy.sum(selection, axis=1) == np.ones(course_count)  
  constraints.append(one_for_each_course_constraint)

  one_timeslot_per_course_no_conflicts = cvxpy.matmul(stage_one_res,selection) <= np.ones((prof_count, time_slot_count)) 
  constraints.append(one_timeslot_per_course_no_conflicts)

  course_costs_constraint =  cvxpy.sum(cvxpy.multiply(selection, time_slot_costs) , axis=1) == course_costs 
  constraints.append(course_costs_constraint) 

  if s2_type == 'baseline':
    max_courses_per_timeslot_constraint = cvxpy.sum(selection, axis=0) <= 6
    constraints.append(max_courses_per_timeslot_constraint)
  
    min_courses_per_timeslot_constraint = cvxpy.sum(selection, axis=0) >= 2
    constraints.append(min_courses_per_timeslot_constraint)

  elif s2_type =='preferences_no_min': 
    max_courses_per_timeslot_constraint = cvxpy.sum(selection, axis=0) <= 6
    constraints.append(max_courses_per_timeslot_constraint)
  
  elif s2_type == 'preferences_no_max': 
    min_courses_per_timeslot_constraint = cvxpy.sum(selection, axis=0) >= 2
    constraints.append(min_courses_per_timeslot_constraint)
  elif s2_type == 'preferences_no_min_no_max': 
    pass 
  else: 
    print('Invalid Stage 2 Type')
    return 


  # Conflict Prevention Constraints 

  # # MWF / MW / WF conflict handling
  for i in range(course_count):
    for j in range(7): 
      # print(i,j)
      new_cons = selection[i][j] + selection[i][j+7] + selection[i][j+14] <= 1
      constraints.append(new_cons) 

  # M T W R F single night course conflict handling 

  # M
  for i in range(course_count): 
    constraints.append(selection[i][6] + selection[i][13] + selection[i][28]<=1)

  # W
  for i in range(course_count): 
    constraints.append(selection[i][6] + selection[i][13] + selection[i][20] + selection[i][30]<=1)

  # # F
  for i in range(course_count): 
    constraints.append(selection[i][6] + selection[i][20] + selection[i][32]<=1)


  # T and R
  for i in range(course_count): 
    constraints.append(selection[i][27] + selection[i][29]<=1)
    constraints.append(selection[i][27] + selection[i][31]<=1)

  cost = cvxpy.sum(cvxpy.multiply(preference_weights, selection)) 

  problem = cvxpy.Problem(cvxpy.Maximize(cost), constraints=constraints)
  opt_cost = problem.solve(solver=cvxpy.GLPK_MI)
  

  return problem, opt_cost, selection.value

In [43]:
def stage_three(stage_two_res, room_slot_costs, course_size_costs): 
  selection  = cvxpy.Variable((room_slot_costs.shape), boolean=True)

  constraints = [] 

  course_count = stage_two_res.shape[0]
  time_slot_count = stage_two_res.shape[1]

  room_slot_count = room_slot_costs.shape[0] 


  # Constraints  
  one_for_each_course_constraint = cvxpy.sum(selection, axis=0) == np.ones(course_count) 
  constraints.append(one_for_each_course_constraint)

  course_costs_constraint =  cvxpy.sum(cvxpy.multiply(selection, room_slot_costs) , axis=0) >= course_size_costs
  constraints.append(course_costs_constraint) 

  one_timeslot_per_course_no_conflicts = cvxpy.matmul(selection, stage_two_res) <= np.ones((room_slot_count, time_slot_count)) 
  constraints.append(one_timeslot_per_course_no_conflicts) 

  
  # Objective function and evaluation

  cost = cvxpy.sum(cvxpy.multiply(np.ones(selection.shape), selection))
  problem = cvxpy.Problem(cvxpy.Minimize(cost), constraints=constraints)
  opt_cost = problem.solve(solver=cvxpy.GLPK_MI)
  
  return problem, opt_cost, selection.value

In [44]:
# Load the dictionary of time slots for lookup purposes

def load_time_slot_dict(): 
  time_slot_dict = {} 

  #MWF Classes
  time_slot_dict[0] = ['MWF', '08:35-9:50']
  time_slot_dict[1] = ['MWF', '10:00-11:15']
  time_slot_dict[2] = ['MWF', '11:15-12:40']
  time_slot_dict[3] = ['MWF', '13:50-15:05']
  time_slot_dict[4] = ['MWF', '15:15-16:30']
  time_slot_dict[5] = ['MWF', '17:00-18:15']
  time_slot_dict[6] = ['MWF', '18:25-19:40']

  #MW Classes
  time_slot_dict[7] = ['MW', '08:35-9:50']
  time_slot_dict[8] = ['MW', '10:00-11:15']
  time_slot_dict[9] = ['MW', '11:15-12:40']
  time_slot_dict[10] = ['MW', '13:50-15:05']
  time_slot_dict[11] = ['MW', '15:15-16:30']
  time_slot_dict[12] = ['MW', '17:00-18:15']
  time_slot_dict[13] = ['MW', '18:25-19:40']

  #WF Classes
  time_slot_dict[14] = ['WF', '08:35-9:50']
  time_slot_dict[15] = ['WF', '10:00-11:15']
  time_slot_dict[16] = ['WF', '11:15-12:40']
  time_slot_dict[17] = ['WF', '13:50-15:05']
  time_slot_dict[18] = ['WF', '15:15-16:30']
  time_slot_dict[19] = ['WF', '17:00-18:15']
  time_slot_dict[20] = ['WF', '18:25-19:40']

  #Tues/Thurs Classes
  time_slot_dict[21] = ['TR', '08:35-9:50']
  time_slot_dict[22] = ['TR', '10:00-11:15']
  time_slot_dict[23] = ['TR', '11:15-12:40']
  time_slot_dict[24] = ['TR', '13:50-15:05']
  time_slot_dict[25] = ['TR', '15:15-16:30']
  time_slot_dict[26] = ['TR', '17:00-18:15']
  time_slot_dict[27] = ['TR', '18:25-19:40']

  #2.5 hour classes throughout the week
  time_slot_dict[28] = ['M', '18:25-21:05']
  time_slot_dict[29] = ['T', '18:25-21:05']
  time_slot_dict[30] = ['W', '18:25-21:05']
  time_slot_dict[31] = ['R', '18:25-21:05']
  time_slot_dict[32] = ['F', '18:25-21:05']
  time_slot_dict[33] = ['S', '09:00-11:40']
  time_slot_dict[34] = ['S', '13:00-15:40']

  return time_slot_dict


# Load the dictionary of rooms for lookup purposes

def load_rooms_dict(): 
  rooms_dict = {} 

  rooms_dict[0] = [1 , "RE 102"]
  rooms_dict[1] = [2, "RE 106"]
  rooms_dict[2] = [2, "RE 122"]
  rooms_dict[3] = [2, "RE 124"]
  rooms_dict[4] = [2, "RE 121"]
  rooms_dict[5] = [2, "RE 119"]
  rooms_dict[6] = [1, "RE 103"]
  rooms_dict[7] = [3, "RE 104"]

  rooms_dict[8] = [2, "RE 242"]
  rooms_dict[9] = [2, "RE 258"]

  rooms_dict[10] = [2, "RE 036"]
  rooms_dict[11] = [2, "RE 032"]
  rooms_dict[12] = [2, "RE 026"]
  rooms_dict[13] = [2, "RE 025"]
  rooms_dict[14] = [2, "RE 027"]

  #Pritzker
  rooms_dict[15] = [2, "PS 121"]
  rooms_dict[16] = [2, "PS 129"]
  rooms_dict[17] = [3, "PS 111"]
  rooms_dict[18] = [2, "PS 152"]
  rooms_dict[19] = [2, "PS 240"]
  rooms_dict[20] = [2, "PS 213"]

  #Stuart
  rooms_dict[21] = [3, "SB 104"]
  rooms_dict[22] = [3, "SB 111"]
  rooms_dict[23] = [3, "SB 113"]
  rooms_dict[24] = [2, "SB 225"]
  rooms_dict[25] = [2, "SB 213"]


  return rooms_dict


In [45]:
def process_data(file_name): 
  # Read in the .xls data file and process the data into the required input matrices

  df = pd.read_excel(file_name)
  df['Course'] = df['Course Subject']+df['Course Number'].astype('str')
  input = df[['Course Code','Course', 'Course Title','Day counts','Room size needed']]

  num_course_sections = input.shape[0]

  prof_dict = {} 

  course_lst_index = {}
  prof_lst_index = {}

  teaching_availability_dict = {} 

  for index, row in df.iterrows():
    prof = row['Primary Instructor']  

    course = row['Course Subject'] + str(row['Course Number'])

    c_len = len(course_lst_index)
    p_len = len(prof_lst_index)

    if course not in course_lst_index.keys(): course_lst_index[course] = c_len 
    if prof not in prof_lst_index.keys(): prof_lst_index[prof] = p_len 

    if prof in prof_dict.keys(): 
      prof_dict[prof].append(course )
    else: 
      prof_dict[prof] = [course]

    if course in teaching_availability_dict.keys(): 
      if prof not in teaching_availability_dict[course]: 
        teaching_availability_dict[course].append(prof)
    else: 
      teaching_availability_dict[course] = [prof]

  prof_lst_index_reverse = {v: k for k, v in prof_lst_index.items()}
  course_lst_index_reverse =  {v: k for k, v in course_lst_index.items()} 

  num_profs = len(prof_lst_index)
  prof_course_teaching_availability = np.zeros((num_profs, num_course_sections))

  for i in range(num_profs): 
    for j in range(num_course_sections):
      prof = prof_lst_index_reverse[i] 
      course =  input['Course'][j]
      
      if prof in teaching_availability_dict[course]: 
        prof_course_teaching_availability[i][j] = 1

  course_costs = input['Day counts'].values


  time_slot_dict = load_time_slot_dict()

  num_time_slots = len(time_slot_dict)

  time_slot_costs = np.zeros((num_course_sections, num_time_slots))

  for i in range(num_time_slots): 
    time_slot_costs[:,i] = len(time_slot_dict[i][0])*np.ones(time_slot_costs[:,i].shape[0])


  rooms_dict = load_rooms_dict()

  num_rooms =  len(rooms_dict)
  room_slot_costs = np.zeros((num_course_sections,num_rooms))


  for i in range(num_rooms): 
    room_slot_costs[:,i] = rooms_dict[i][0]*np.ones(room_slot_costs[:,i].shape[0])

  room_slot_costs = room_slot_costs.T

  

  course_size_costs = input['Room size needed'].values

  # Stage 1 input: prof_course_teaching_availability
  # Stage 2 input: stage 1 result, time_slot_costs, course_costs
  # Stage 3 input: stage 2 result,  room_slot_costs, course_size_costs 

  return df, prof_lst_index_reverse, course_lst_index_reverse,  prof_course_teaching_availability,  time_slot_costs, course_costs, course_size_costs, room_slot_costs

In [46]:
def parse_results(df, prof_lst_index_reverse, stage_one_result, stage_two_result, stage_three_result,output_name, output=False): 
  res = [] 

  time_slot_dict = load_time_slot_dict()
  rooms_dict = load_rooms_dict()

  for i in range(stage_one_result.shape[0]):  #number of profs
    
    for j in range(stage_one_result.shape[1]):  #number of courses 
      
    
      if stage_one_result[i][j] == 1: 
        tmp_lst = [] 
        tmp_lst.append(prof_lst_index_reverse[i])
        tmp_lst.append(df['Course Code'][j]) 
  
        for k in range(stage_two_result.shape[1]): # number of time slots
          if stage_two_result[j][k] == 1: 
            tmp_lst.append(time_slot_dict[k][0])
            tmp_lst.append(time_slot_dict[k][1])

        # print(stage_three_result.shape[1])
        for k in range(stage_three_result.shape[0]): # number of room slots
          if stage_three_result[k][j] == 1: 
            tmp_lst.append(rooms_dict[k][1])
            res.append(tmp_lst)

  res_df = pd.DataFrame(res, columns=['Instructor', 'Course', 'Days','Time','Room'])
  if output: res_df.to_csv('{}.csv'.format(output_name))
  return res_df

In [47]:
def load_weights(prof_file, student_file,stage_one_result): 

  from numpy import genfromtxt
  prof_weights = genfromtxt(prof_file, delimiter=',')
  student_weights = genfromtxt(student_file, delimiter=',')#*12)



  weights = np.multiply(prof_weights, np.tile(student_weights,(stage_one_result.shape[0],1)))

  input_weights = np.matmul(stage_one_result.T, weights)

  return input_weights

In [48]:
def run_model_unweighted(file_name,stage2_type, output_name, output=False): 
  df, prof_lst_index_reverse, course_lst_index_reverse,  prof_course_teaching_availability,time_slot_costs, course_costs, course_size_costs, room_slot_costs = process_data(file_name)
  problem1, opt1, stage_one_result = stage_one(prof_course_teaching_availability)

  problem2, opt2, stage_two_result = stage_two(stage_one_result, time_slot_costs, course_costs, preference_weights = np.ones(time_slot_costs.shape), s2_type=stage2_type )

  problem3, opt3, stage_three_result = stage_three(stage_two_result, room_slot_costs, course_size_costs)

  res_df = parse_results(df,prof_lst_index_reverse, stage_one_result, stage_two_result, stage_three_result,output_name, output)

  return res_df, opt1,opt2,opt3

In [49]:
def run_model_weighted(file_name,prof_weights_file, student_weights_file, stage2_type,output_name, output=False): 
  df, prof_lst_index_reverse, course_lst_index_reverse,  prof_course_teaching_availability,time_slot_costs, course_costs, course_size_costs, room_slot_costs = process_data(file_name)
  problem1, opt1, stage_one_result = stage_one(prof_course_teaching_availability)

  weights = load_weights(prof_weights_file, student_weights_file,stage_one_result)
  problem2, opt2, stage_two_result = stage_two(stage_one_result, time_slot_costs, course_costs, preference_weights = weights,s2_type=stage2_type )

  problem3, opt3, stage_three_result = stage_three(stage_two_result, room_slot_costs, course_size_costs)

  res_df = parse_results(df, prof_lst_index_reverse, stage_one_result, stage_two_result, stage_three_result,output_name, output)

  return res_df, opt1,opt2,opt3

In [50]:
def run_model_weighted_stage1stage2(file_name, prof_weights_file, student_weights_file, stage2_type, output_name, output=False): 
  df, prof_lst_index_reverse, course_lst_index_reverse,  prof_course_teaching_availability,time_slot_costs, course_costs, course_size_costs, room_slot_costs = process_data(file_name)
  problem1, opt1, stage_one_result = stage_one(prof_course_teaching_availability)

  weights = load_weights(prof_weights_file, student_weights_file,stage_one_result)
  problem2, opt2, stage_two_result = stage_two(stage_one_result, time_slot_costs, course_costs, preference_weights = weights, s2_type=stage2_type)

  res_df = parse_results_s1s2(df, prof_lst_index_reverse, stage_one_result, stage_two_result,output_name, output)

  return res_df, opt1,opt2

In [51]:
def parse_results_s1s2(df, prof_lst_index_reverse, stage_one_result, stage_two_result,output_name, output=False): 
  res = [] 

  time_slot_dict = load_time_slot_dict()
  rooms_dict = load_rooms_dict()

  for i in range(stage_one_result.shape[0]):  #number of profs
    
    for j in range(stage_one_result.shape[1]):  #number of courses 
      
    
      if stage_one_result[i][j] == 1: 
        tmp_lst = [] 
        tmp_lst.append(prof_lst_index_reverse[i])
        tmp_lst.append(df['Course Code'][j]) 
  
        for k in range(stage_two_result.shape[1]): # number of time slots
          if stage_two_result[j][k] == 1: 
            tmp_lst.append(time_slot_dict[k][0])
            tmp_lst.append(time_slot_dict[k][1])
            res.append(tmp_lst) 
            

  res_df = pd.DataFrame(res, columns=['Instructor', 'Course', 'Days','Time'])
  if output: res_df.to_csv('{}.csv'.format(output_name))
  return res_df

Running the Model

Model 1

In [61]:
from google.colab import files
file_upload = files.upload()

Saving Test data 1.xlsx to Test data 1 (5).xlsx


In [62]:
model1_res, opt1,opt2,opt3 = run_model_unweighted('Test data 1.xlsx',
                    stage2_type='baseline',                                      
                   output_name = 'model1_baseline_no_weights',  
                   output=False)

print('Utils for Model 1:',opt2)

KeyError: ignored

Looking at the model from 1 

In [57]:
model1_res

NameError: ignored

Model 2: Prefrence 

In [ ]:
model2_res, opt1,opt2,opt3 = run_model_weighted('CourseStatusReport_Edited.xls', 
                  'MATH 380 Project Model Weights Faculty.csv',
                   'MATH 380 Project Model Weights Student.csv',
                   stage2_type='baseline',
                   output_name = 'model2_baseline_with_weights', 
                   output=False)
print('Utils for Model 2:',opt2)

In [ ]:
model2_res